# Driving Bot

Before doing anything, copy your `RANDOM_STRING` from the previous notebook into the code below (all the way at the bottom), and run the code to save your Airtable URL just like you did in the previous notebook.

In [ ]:
##########################################
###DRIVING NOTEBOOK INITIALIZATION CODE###
##########################################

##IMPORTS##
from easygopigo3 import EasyGoPiGo3
import numpy as np
import time
import json
import requests

# Initialize easy gpg
easyGPG = EasyGoPiGo3()
# Make sure that all sensors and output devices are uncofigured to start
easyGPG.reset_all()
easyGPG.reset_speed()

print('Robot initialized.')

###FUNCTIONS###
# get the Airtable API Key from a local file
def getKey(filename):
    fin = open(filename)
    for line in fin:
        key = line.strip()
    return key

# delete all data in an airtable
def clear_table(url):
    response = requests.get(url)
    data = json.loads(response.text)['records']
    
    YOUR_API_KEY=getKey('api_key.txt')
    to_strip = "?api_key="+YOUR_API_KEY
    stripped_url = url.strip(to_strip) + '/'
    
    for entry in data:
        ID = entry['id']
        del_url = stripped_url+ID+to_strip
        #print(del_url)
        response2=requests.delete(del_url)
        #print(response2.text)
    
    return
    
#Driving function that we'll use later
def drive_and_post(url, heading, location1, location2):
    
    ###MATH###
    #Use the distance formula to find the distance to drive
    dx=location2[0]-location1[0] 
    dy=location2[1]-location1[1] 
    dist = np.sqrt((dx)**2 + (dy)**2) 
    
    #Use the arctan function to find the abs angle to turn
    if dx>0:
        abs_angle = (np.arctan((dy/dx)))*180/(np.pi)
    elif dx<0:
        abs_angle = 180+(np.arctan((dy/dx)))*180/(np.pi)
    else:
        if dy>0:
            abs_angle=90
        else:
            abs_angle=-90
            
    
    ###TURNING###   
    easyGPG.reset_encoders() #reset encoders so we know how far we have driven
    
    angle=abs_angle-heading #angle to turn is absolute angle - heading angle
    easyGPG.turn_degrees(-angle) #turn 
    heading=abs_angle #set heading to where we are facing now
    
    if -angle>=0:
        encoders=easyGPG.read_encoders()
        post_values(url,(1,encoders[0],encoders[1]))
    elif -angle<0:
        encoders=easyGPG.read_encoders()
        post_values(url,(-1,encoders[0],encoders[1]))
    
    ###DRIVING###
    easyGPG.reset_encoders() #reset encoders so we know how far we have driven
    
    easyGPG.set_speed(300) #set speed 
    easyGPG.drive_cm(dist) #begin driving
    
    encoders=easyGPG.read_encoders()
    post_values(url,(0,encoders[0],encoders[1]))
    
    ###RETURN###
    return(heading)


'''---------------------------------------------------------'''
'''Put your random string in the location below, just like last time.'''

YOUR_API_KEY=getKey('api_key.txt')
BASETABLENAME = 'driving_bot'
'''---------------------------------------------------------'''
RANDOM_STRING = "appqTlRwTmBGwlBxK"#Put your random string here as shown above
'''---------------------------------------------------------'''

url = 'https://api.airtable.com/v0/'+RANDOM_STRING+'/'+BASETABLENAME+"?api_key="+YOUR_API_KEY

print(url)
print('Type the url above into your browser and ensure it takes you to your airtable\'s data')

**Ok, let's get started.**

The driving bot's job will be to drive along a predetermined path, and log the types of moves it makes and the corresponding encoder values to airtable. There will be three types of moves, denoted by three numbers. 

- `move_type = 0` means that the robot drove forward
- `move_type = 1` means that the robot turned right
- `move_type = -1` means that the robot turned left

For each of these moves we will log the values of the encoders after the move, namely: `left_encoder` and `right_encoder`

## Posting values

Let's make a function called `post_values(url, driving_data)` that takes a url to post to, and data set of values as a tuple [(link to what a tuple is since we haven't covered it that much in Python)](https://www.w3schools.com/python/python_tuples.asp). and then posts the values in that tuple to Airtable.

Our `driving_data` tuple will look like this:

`driving_data = (move_type, left_encoder, right_encoder)` 

An example driving data could be: `driving_data = (0, 200, 200)` which means the robot drove forward for 200 turns of the encoder on both motors. Remember, if we wanted to get the left encoder value from this tuple we would index it like so: 

`left_encoder = driving_data[1]`

Finish the code below to save the `move_type, left_encoder, right_encoder` values from the `driving_data` input and then post those values to Airtable by inserting them into the `payload` variable in their corresponding places.

In [ ]:
def post_values(url, driving_data):
    
    '''--------------------------------------------'''
    '''
    In the space below:
    -Get move_type, left_encoder, and right_encoder from driving_data
    -Save them into the correct slots in payload
    '''
    
    move_type,left_encoder,right_encoder = driving_data #get our values from the tuple
    
    payload={
      "fields": {
        "move_type": move_type, #put our values into the payload
        "left_encoder": left_encoder,
        "right_encoder": right_encoder
      }
    }
    
    '''--------------------------------------------'''
    
    #Post the payload to airtable.
    response = requests.post(url, json=payload)
    
    ###This is for debugging in case something breaks. 
    verbose=False
    if verbose:
        print('Response was: ',response.text)

    return

print('Your post_values function has been saved.')

## Try it out

Try out your post values function in the code below, by calling it with `url` (already saved in the computer's memory as you ran it in the initialization script) and your own set of `driving_data` that you define!

Take a look at your Airtable: did your values show up? If so, we're ready to start driving!

In [ ]:
#Try out post_values!
driving_data=(1,2,3) #enter some test values here
post_values(url,driving_data) #call your function!
print('Values posted. Check Airtable to see if they updated!')

## Driving and posting

We've created a `drive_and_post` function for you, that uses the `post_values` function you already created to drive to different points specified in a variable called `MAP` (much like the orienteering notebook we did last week) and log its progress from its encoders as it drives. You can scroll back to the top of the page to take a look at how the function works. 

Fill out the `MAP` variable with points you'd like to have your robot drive to (don't tell your partner where your robot is going!). Run the code below and see what it does. Check your airtable after to see your how your robot logged its motion!

In [ ]:
###DRIVE AND POST CODE###

###VARIABLES###
heading=0
'''----------------------------------------'''
##Your orienteering map!##
'''Your robot will drive to the points on this map sequentially,
to add a point, make sure to add a comma after the last one'''
MAP=[
    [0,0], #always start at 0,0
    #point format:
    #[x,y],
    #the last point should not have a comma
    [50,0],
    [0,0]
]
'''----------------------------------------'''

clear_table(url) #clear the airtable before starting
    
for i in range(len(MAP)-1):
    try:
        heading=drive_and_post(url, heading, MAP[i], MAP[i+1])
    except IndexError:
        break
            
print('All done!')

In [ ]:
#Emergency Stop
easyGPG.stop()